In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from scipy.io import arff
from sklearn.preprocessing import MinMaxScaler
from scipy import stats

import sys
sys.path.insert(0, '..')

from algorithms.fs_neat import FS_NEAT
from algorithms.n3o import N3O
from algorithms.neat import set_seed
from utilities.activation_functions import Gaussian
from utilities.fitness_functions import torch_fitness_function

In [3]:
# Load train dataset
data = arff.loadarff('../datasets/prostate_tumorVSNormal_train.arff')
df_train = pd.DataFrame(data[0])

# Load test dataset
data = arff.loadarff('../datasets/prostate_tumorVSNormal_test.arff')
df_test = pd.DataFrame(data[0])

df_train.head()

,AFFX-MurIL2_at,AFFX-MurIL10_at,AFFX-MurIL4_at,AFFX-MurFAS_at,AFFX-BioB-5_at,AFFX-BioB-M_at,AFFX-BioB-3_at,AFFX-BioC-5_at,AFFX-BioC-3_at,AFFX-BioDn-5_at,...,101_at,102_at,103_at,104_at,105_at,106_at,107_at,108_g_at,109_at,Class
0,-9.0,1.0,1.0,15.0,-2.0,-3.0,4.0,8.0,-12.0,-12.0,...,5.0,3.0,21.0,15.0,1.0,0.0,14.0,-23.0,14.0,b'Tumor'
1,-2.0,1.0,1.0,4.0,-2.0,-5.0,0.0,8.0,-5.0,-9.0,...,3.0,2.0,12.0,7.0,4.0,5.0,4.0,-10.0,25.0,b'Tumor'
2,-6.0,17.0,6.0,29.0,4.0,-11.0,-8.0,10.0,-24.0,-32.0,...,-5.0,11.0,11.0,36.0,8.0,7.0,21.0,-82.0,10.0,b'Tumor'
3,0.0,9.0,4.0,19.0,-10.0,-18.0,-18.0,5.0,-33.0,-31.0,...,-7.0,6.0,9.0,22.0,3.0,20.0,-7.0,-62.0,24.0,b'Tumor'
4,-1.0,0.0,1.0,5.0,0.0,-4.0,1.0,6.0,-4.0,-9.0,...,6.0,1.0,140.0,10.0,3.0,8.0,8.0,-27.0,20.0,b'Tumor'


In [4]:
# Change category class label to binary class label
labels = {b'Tumor' : 1, b'Normal' : 0}
df_train['Class'] = df_train['Class'].replace(labels)
df_test['Class'] = df_test['Class'].replace(labels)

# Count class distribution from both datasets
n_tumor_train = np.sum(df_train['Class'].to_numpy(dtype=np.float32))
n_normal_train = df_train.shape[0] - n_tumor_train
n_tumor_test = np.sum(df_test['Class'].to_numpy(dtype=np.float32))
n_normal_test = df_test.shape[0] - n_tumor_test

# Print information
print(f"Train dataset shape: {df_train.shape}, Relapsed instances: {n_tumor_train}, Non-Relapsed instances: {n_normal_train}")
print(f"Test dataset shape: {df_test.shape}, Relapsed instances: {n_tumor_test}, Non-Relapsed instances: {n_normal_test}")

Train dataset shape: (102, 12601), Relapsed instances: 52.0, Non-Relapsed instances: 50.0
Test dataset shape: (34, 12601), Relapsed instances: 25.0, Non-Relapsed instances: 9.0


In [5]:
# Convert train dataset to Numpy array
x_train = df_train.iloc[:, :-1].to_numpy(dtype=np.float32)
y_train = df_train.iloc[:, -1].to_numpy(dtype=np.float32)

# Convert test dataset to Numpy array
x_test = df_test.iloc[:, :-1].to_numpy(dtype=np.float32)
y_test = df_test.iloc[:, -1].to_numpy(dtype=np.float32)

# Kruskal Wallis H Test
kw_pvalue = np.zeros(x_train.shape[1])

for feature in range(x_train.shape[1]):
	_, kw_pvalue[feature] = stats.kruskal(x_train[:, feature], y_train)

kw_feature_selected = np.argwhere(kw_pvalue < 1e-6)
kw_pvalue = kw_pvalue[kw_feature_selected]
x_train_kw = x_train[:, kw_feature_selected[:, 0]]
x_test_kw = x_test[:, kw_feature_selected[:, 0]]

print(f'Attributes selected after KW H Test: {kw_feature_selected.shape[0]}')


Attributes selected after KW H Test: 9735
